In [1]:
import logging
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s', level=logging.INFO)

In [2]:
def get_img_size(img_uri):
    import requests
    from PIL import Image
    
    #logging.info(img_uri)
    im = Image.open(requests.get(img_uri, stream=True).raw)
    width = im.size[0]
    height = im.size[1]
    
    return width, height

In [3]:
def detect_labels_uri_json(uri):
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    
    image.source.image_uri = uri
    
    response = client.label_detection(image=image)
    labels = response.label_annotations
    
    # convert the result into json
    from google.cloud.vision_v1 import AnnotateImageResponse
    response_json = AnnotateImageResponse.to_json(response)
    
    return response_json

In [4]:
def detect_color_json(uri):
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri
    
    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    
    # convert the result into json
    from google.cloud.vision_v1 import AnnotateImageResponse
    response_json = AnnotateImageResponse.to_json(response)
    
    return response_json

In [5]:
def insert_into_mysql(sql_database, sql_table, article_id, info):
    import mysql.connector
    
    sql_db = mysql.connector.connect(
        host = '127.0.0.1',
        user = 'root',
        password = '',
        database = sql_database
    )
    cursor = sql_db.cursor()
    
    # to check whether TABLE is in the DATABASE
    cursor.execute("SHOW TABLES LIKE '{}'".format(sql_table))
    temp_result = cursor.fetchone()
    if (temp_result):
        logging.info("TABLE {} exists in DATABASE {}".format(sql_table, sql_database))
    else:
        logging.info("NO TABLE {} in DATABASE {}".format(sql_table, sql_database))
        
    sql_update = 'UPDATE {} SET img_width_pixel=%s, img_height_pixel=%s, img_labels=%s, img_colors=%s WHERE id={}'.format(sql_table, article_id)
    #print(sql_update)
    cursor.execute(sql_update, info)
    logging.info(str(article_id)+" UPDATED")

In [42]:
def detect_labels_json_file(path):
    from google.cloud import vision
    import io
    
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    
    response = client.label_detection(image=image)
    labels = response.label_annotations
    
    # convert the result into json
    from google.cloud.vision_v1 import AnnotateImageResponse
    response_json = AnnotateImageResponse.to_json(response)
    
    return response_json
lables_json = detect_labels_json_file("{}_0.jpg".format(38325))
print(lables_json)

{
  "error": {
    "code": 3,
    "message": "Bad image data.",
    "details": []
  },
  "faceAnnotations": [],
  "landmarkAnnotations": [],
  "logoAnnotations": [],
  "labelAnnotations": [],
  "localizedObjectAnnotations": [],
  "textAnnotations": []
}


In [20]:
def detect_color_json_file(path):
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()
    image = vision.Image()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    
    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    
    # convert the result into json
    from google.cloud.vision_v1 import AnnotateImageResponse
    response_json = AnnotateImageResponse.to_json(response)
    
    return response_json

In [43]:
# import ad dataset from csv
import pandas as pd
import numpy as np
import time

ad = pd.read_csv('C:/Users/Public/output.csv')
ad_article_id = ad['article_id']
ad_article_id = ad_article_id.unique()
ad_article_id = np.sort(ad_article_id)
#ad_article_id = ad_article_id[:3]
print(len(ad_article_id))
# ad_article_id_v2 = []
# for i in ad_article_id:
#     #if i>=8647:
#     if i==57924:
#         ad_article_id_v2.append(i)
ad_article_id_v2 = [12006, 32912, 32947, 37014, 38325]
print(len(ad_article_id_v2))
img_info_list = []
error_article_id = []

#for article_id in ad_article_id:
for article_id in ad_article_id_v2:
    try:
        logging.info("{} START".format(article_id))

        # create uri with domain
        img_uri = "http://taiwanlikes.itaiwan.net/uploaded_images/{}_0.jpg".format(article_id)

        # get size (by pixel)
        width, height = get_img_size(img_uri)
        logging.info("width: {}, height: {}".format(width, height))

        import os
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/ching870423/extract_text_practice/information/contentanalysis-0193c6e708c5.json"

        # get label info (by json)
        if article_id==0:
            lables_json = detect_labels_json_file("{}_0.jpg".format(article_id))
        else:
            labels_json = detect_labels_uri_json(img_uri)
        logging.info("{} labels DONE".format(article_id))

        # get color info (by json)
        if article_id==0:
            colors_json = detect_color_json_file("{}_0.jpg".format(article_id))
        else:
            colors_json = detect_color_json(img_uri)
        logging.info("{} colors DONE".format(article_id))

        img_article_info = (width, height, labels_json, colors_json)
        img_info_list.append(img_article_info)
        sql_database = "contentanalysis"
        sql_table = "articles"
        insert_into_mysql(sql_database, sql_table, article_id, img_article_info)

        logging.info(img_uri+" DONE")

        # take a break
        if article_id%100 == 0:
            logging.info("SLEEP for 2 seconds...Zzz...")
            time.sleep(2)
            
    except:
        logging.info("{} error".format(article_id))
        error_article_id.append(article_id)
        continue

logging.info("Finished")
print("Error article id")
print(error_article_id)
print(img_info_list[:3])

2021-08-05 08:55:10,283: INFO: 38325 START


23834
13


2021-08-05 08:55:11,045: INFO: width: 640, height: 518
2021-08-05 08:55:13,799: INFO: 38325 labels DONE
2021-08-05 08:55:16,019: INFO: 38325 colors DONE
2021-08-05 08:55:16,025: INFO: TABLE articles exists in DATABASE contentanalysis
2021-08-05 08:55:16,028: INFO: 38325 UPDATED
2021-08-05 08:55:16,030: INFO: http://taiwanlikes.itaiwan.net/uploaded_images/38325_0.jpg DONE
2021-08-05 08:55:16,032: INFO: 32912 START
2021-08-05 08:55:16,776: INFO: width: 600, height: 812
2021-08-05 08:55:18,912: INFO: 32912 labels DONE
2021-08-05 08:55:20,124: INFO: 32912 colors DONE
2021-08-05 08:55:20,131: INFO: TABLE articles exists in DATABASE contentanalysis
2021-08-05 08:55:20,133: INFO: 32912 UPDATED
2021-08-05 08:55:20,135: INFO: http://taiwanlikes.itaiwan.net/uploaded_images/32912_0.jpg DONE
2021-08-05 08:55:20,136: INFO: 12006 START
2021-08-05 08:55:20,690: INFO: width: 700, height: 700
2021-08-05 08:55:22,503: INFO: 12006 labels DONE
2021-08-05 08:55:23,226: INFO: 12006 colors DONE
2021-08-05 08

Error article id
[]
[(640, 518, '{\n  "error": {\n    "code": 3,\n    "message": "Bad image data.",\n    "details": []\n  },\n  "faceAnnotations": [],\n  "landmarkAnnotations": [],\n  "logoAnnotations": [],\n  "labelAnnotations": [],\n  "localizedObjectAnnotations": [],\n  "textAnnotations": []\n}', '{\n  "error": {\n    "code": 3,\n    "message": "Bad image data.",\n    "details": []\n  },\n  "faceAnnotations": [],\n  "landmarkAnnotations": [],\n  "logoAnnotations": [],\n  "labelAnnotations": [],\n  "localizedObjectAnnotations": [],\n  "textAnnotations": []\n}'), (600, 812, '{\n  "error": {\n    "code": 3,\n    "message": "Bad image data.",\n    "details": []\n  },\n  "faceAnnotations": [],\n  "landmarkAnnotations": [],\n  "logoAnnotations": [],\n  "labelAnnotations": [],\n  "localizedObjectAnnotations": [],\n  "textAnnotations": []\n}', '{\n  "error": {\n    "code": 3,\n    "message": "Bad image data.",\n    "details": []\n  },\n  "faceAnnotations": [],\n  "landmarkAnnotations": [],\

In [9]:
print(len(error_article_id))
print(error_article_id)

561
[8647, 9037, 9109, 9444, 9900, 10194, 10874, 10897, 11008, 11017, 11030, 11255, 11304, 11496, 11602, 11612, 11617, 11636, 11771, 11859, 12127, 23841, 27003, 27900, 27901, 28521, 28561, 28584, 29760, 29842, 29861, 30553, 30610, 31062, 31215, 31437, 31493, 31494, 31956, 32069, 32114, 32169, 32211, 32289, 32546, 32573, 36428, 37297, 38241, 39248, 43737, 45339, 46719, 47551, 47693, 50982, 51724, 52489, 52624, 52712, 52746, 52822, 52841, 52843, 52850, 52851, 52870, 52885, 52908, 52920, 52926, 52942, 52960, 52986, 52988, 52990, 52999, 53002, 53004, 53005, 53007, 53008, 53010, 53021, 53023, 53025, 53067, 53069, 53097, 53098, 53099, 53109, 53110, 53121, 53201, 53240, 53275, 53306, 53317, 53319, 53454, 53455, 53461, 53463, 53492, 53505, 53544, 53546, 53548, 53566, 53572, 53587, 53599, 53613, 53649, 53650, 53658, 53659, 53707, 53759, 53762, 53763, 53764, 53765, 53767, 53769, 53770, 53771, 53772, 53774, 53775, 53794, 53795, 53796, 53797, 53798, 53809, 53813, 53815, 53816, 53822, 53831, 53867,

# test

In [9]:
# to read iamge from url
import requests
from PIL import Image
test_id = []
for p in ad_article_id.head(5):
    test_id.append(p)
# remove duplicate & append a customize value id
test_id = list(dict.fromkeys(test_id))
test_id.append("98567")

test_img_url = []
for i in test_id:
    url = "http://taiwanlikes.itaiwan.net/uploaded_images/{}_0.jpg".format(i)
    test_img_url.append(url)

## get size

In [11]:
for url in test_img_url:
    print(url)
    im = Image.open(requests.get(url, stream=True).raw)
    print(im.size) # size is a (width, height) tuple.

http://taiwanlikes.itaiwan.net/uploaded_images/42303_0.jpg
(640, 427)
http://taiwanlikes.itaiwan.net/uploaded_images/37119_0.jpg
(318, 380)
http://taiwanlikes.itaiwan.net/uploaded_images/42474_0.jpg
(510, 573)
http://taiwanlikes.itaiwan.net/uploaded_images/37016_0.jpg
(580, 773)
http://taiwanlikes.itaiwan.net/uploaded_images/98567_0.jpg
(800, 420)


##  get color
https://cloud.google.com/vision/docs/detecting-properties?hl=zh-TW

In [46]:
def detect_color(uri):
    from google.cloud import vision
    from google.cloud.vision_v1 import AnnotateImageResponse
    import io
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri
    
    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    #print(props)
    response_json = AnnotateImageResponse.to_json(response)
    return response_json, props
    
#     print("Properties:")
#     for color in props.dominant_colors.colors:
#         print('fraction: {}'.format(color.pixel_fraction))
#         print('\tr: {}'.format(color.color.red))
#         print('\tg: {}'.format(color.color.green))
#         print('\tb: {}'.format(color.color.blue))
#         print('\ta: {}'.format(color.color.alpha))

In [48]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/ching870423/extract_text_practice/information/contentanalysis-0193c6e708c5.json"
#url = "http://taiwanlikes.itaiwan.net/uploaded_images/98567_0.jpg"
url = "http://taiwanlikes.itaiwan.net/uploaded_images/42474_0.jpg"
print(url)
props_json, props = detect_color(url)
print(type(props_json))
print(props_json)

http://taiwanlikes.itaiwan.net/uploaded_images/42474_0.jpg
<class 'str'>
{
  "imagePropertiesAnnotation": {
    "dominantColors": {
      "colors": [
        {
          "color": {
            "red": 143.0,
            "green": 162.0,
            "blue": 193.0
          },
          "score": 0.16816781,
          "pixelFraction": 0.09770475
        },
        {
          "color": {
            "red": 104.0,
            "green": 75.0,
            "blue": 81.0
          },
          "score": 0.09434084,
          "pixelFraction": 0.09839458
        },
        {
          "color": {
            "red": 205.0,
            "green": 196.0,
            "blue": 191.0
          },
          "score": 0.070643865,
          "pixelFraction": 0.09645052
        },
        {
          "color": {
            "red": 29.0,
            "green": 19.0,
            "blue": 23.0
          },
          "score": 0.054943293,
          "pixelFraction": 0.06409131
        },
        {
          "color": {
      

In [40]:
print(type(props))

<class 'google.cloud.vision_v1.types.image_annotator.ImageProperties'>


## get label
https://cloud.google.com/vision/docs/labels?hl=zh-TW#vision_label_detection-python

In [49]:
def detect_labels_uri(uri):
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    
    image.source.image_uri = uri
    
    response = client.label_detection(image=image)
    labels = response.label_annotations
    
    from google.cloud.vision_v1 import AnnotateImageResponse
    response_json = AnnotateImageResponse.to_json(response)
    #print('Labels:')
    
#     for label in labels:
#         print(label.description)
        
    #print(labels)
    return response_json, labels

In [50]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/ching870423/extract_text_practice/information/contentanalysis-0193c6e708c5.json"
#url = "http://taiwanlikes.itaiwan.net/uploaded_images/98567_0.jpg"
url = "http://taiwanlikes.itaiwan.net/uploaded_images/42474_0.jpg"
print(url)
labels_json, labels = detect_labels_uri(url)
print(type(labels_json))
print(labels_json)

http://taiwanlikes.itaiwan.net/uploaded_images/42474_0.jpg
<class 'str'>
{
  "labelAnnotations": [
    {
      "mid": "/m/0dzct",
      "description": "Face",
      "score": 0.984265,
      "topicality": 0.984265,
      "locale": "",
      "confidence": 0.0,
      "locations": [],
      "properties": []
    },
    {
      "mid": "/m/06z04",
      "description": "Skin",
      "score": 0.9765479,
      "topicality": 0.9765479,
      "locale": "",
      "confidence": 0.0,
      "locations": [],
      "properties": []
    },
    {
      "mid": "/m/0jyfg",
      "description": "Glasses",
      "score": 0.9757826,
      "topicality": 0.9757826,
      "locale": "",
      "confidence": 0.0,
      "locations": [],
      "properties": []
    },
    {
      "mid": "/m/04hgtk",
      "description": "Head",
      "score": 0.974903,
      "topicality": 0.974903,
      "locale": "",
      "confidence": 0.0,
      "locations": [],
      "properties": []
    },
    {
      "mid": "/m/019nj4",
      "de

In [42]:
print(type(labels))

<class 'proto.marshal.collections.repeated.RepeatedComposite'>


In [43]:
from google.protobuf.json_format import MessageToJson
serialized = MessageToJson(labels)
print(type(serialized))
print(serialized)

AttributeError: 'RepeatedComposite' object has no attribute 'DESCRIPTOR'

In [34]:
labels_list = []
for l in labels:
    labels_list.append(l)
print(labels_list)

[mid: "/m/0dzct"
description: "Face"
score: 0.984265
topicality: 0.984265
, mid: "/m/06z04"
description: "Skin"
score: 0.9765479
topicality: 0.9765479
, mid: "/m/0jyfg"
description: "Glasses"
score: 0.9757826
topicality: 0.9757826
, mid: "/m/04hgtk"
description: "Head"
score: 0.974903
topicality: 0.974903
, mid: "/m/019nj4"
description: "Smile"
score: 0.97406685
topicality: 0.97406685
, mid: "/m/0bt9lr"
description: "Dog"
score: 0.90521383
topicality: 0.90521383
, mid: "/m/09ggk"
description: "Purple"
score: 0.8763246
topicality: 0.8763246
, mid: "/m/02wzbmj"
description: "Standing"
score: 0.86421657
topicality: 0.86421657
, mid: "/m/0244x1"
description: "Gesture"
score: 0.85260487
topicality: 0.85260487
, mid: "/m/01lrl"
description: "Carnivore"
score: 0.84202945
topicality: 0.84202945
]


In [35]:
print(type(labels_list))

<class 'list'>


In [37]:
test_label_df = pd.DataFrame(labels_list)
test_label_df
# filename = "test_labels.csv"
# test_label_df.to_csv(filename)

,0
0,"mid: ""/m/0dzct""\ndescription: ""Face""\nscore: 0..."
1,"mid: ""/m/06z04""\ndescription: ""Skin""\nscore: 0..."
2,"mid: ""/m/0jyfg""\ndescription: ""Glasses""\nscore..."
3,"mid: ""/m/04hgtk""\ndescription: ""Head""\nscore: ..."
4,"mid: ""/m/019nj4""\ndescription: ""Smile""\nscore:..."
5,"mid: ""/m/0bt9lr""\ndescription: ""Dog""\nscore: 0..."
6,"mid: ""/m/09ggk""\ndescription: ""Purple""\nscore:..."
7,"mid: ""/m/02wzbmj""\ndescription: ""Standing""\nsc..."
8,"mid: ""/m/0244x1""\ndescription: ""Gesture""\nscor..."
9,"mid: ""/m/01lrl""\ndescription: ""Carnivore""\nsco..."
